In [0]:
# setup drive
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
print('success!')

     |████████████████████████████████| 993kB 3.5MB/s 
success!


In [0]:
!nvidia-smi

Fri May 10 04:13:18 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    16W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
# imports
import torch, pickle, sys, time, random, os
import spacy
from spacy.vocab import Vocab
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import random
import math
import time
import copy
import matplotlib.pyplot as plt
from torchtext import datasets
from torchtext.data import Field, BucketIterator

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [0]:
# tokenizer
!python -m spacy download en
!python -m spacy download de


    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')

     |████████████████████████████████| 38.2MB 71.6MB/s 
  Stored in directory: /tmp/pip-ephem-wheel-cache-iy01iook/wheels/a5/9e/96/785f498e5cea276b337aab8a79259d32a93f03bd744d71eef5
Successfully built de-core-news-sm

    Linking successful
    /usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/de

    You can now load the model via spacy.load('de')



In [0]:
# get the dataset
link = 'https://drive.google.com/open?id=1jOkN_kN5qR62BY8zzq8XOrLy1qxjb8Lp'
fluff, id = link.split('=')
print(id)
f_tr_en = drive.CreateFile({'id': id})
f_tr_en.GetContentFile('./train.en')
link = 'https://drive.google.com/open?id=1_5iLJrtwuEK8ipv62LExuy6iDffyhndG'
fluff, id = link.split('=')
print(id)
f_tr_de = drive.CreateFile({'id': id})
f_tr_de.GetContentFile('./train.de')
link = 'https://drive.google.com/open?id=155XUlxhCHzPHV_TDF8pGFI6v41S6-ssh'
fluff, id = link.split('=')
print(id)
f_dv_en = drive.CreateFile({'id': id})
f_dv_en.GetContentFile('./dev.en')
link = 'https://drive.google.com/open?id=1DidcYbsHBlAf0HrHlN1aSqf0OxOCsUcp'
fluff, id = link.split('=')
print(id)
f_dv_de = drive.CreateFile({'id': id})
f_dv_de.GetContentFile('./dev.de')

1jOkN_kN5qR62BY8zzq8XOrLy1qxjb8Lp
1_5iLJrtwuEK8ipv62LExuy6iDffyhndG
155XUlxhCHzPHV_TDF8pGFI6v41S6-ssh
1DidcYbsHBlAf0HrHlN1aSqf0OxOCsUcp


In [0]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]
  
SRC = Field(tokenize = tokenize_de, 
            init_token = '<bos>', 
            eos_token = '<eos>', 
            unk_token = '<unk>',
            pad_token = '<pad>',
            lower = False)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<bos>', 
            eos_token = '<eos>', 
            unk_token = '<unk>',
            pad_token = '<pad>', 
            lower = False)

In [0]:
# create the training and validation data

data_train = datasets.TranslationDataset(
            path='./train', exts=('.de', '.en'), fields=(SRC, TRG))

data_dev = datasets.TranslationDataset(
            path='./dev', exts=('.de', '.en'), fields=(SRC, TRG))

print(f"Number of training examples: {len(data_train.examples)}")
print(f"Number of dev examples: {len(data_dev.examples)}")

print(vars(data_train.examples[0]))

Number of training examples: 196884
Number of dev examples: 7883
{'src': ['David', 'Gallo', ':', 'Das', 'ist', 'Bill', 'Lange', '.', 'Ich', 'bin', 'Dave', 'Gallo', '.'], 'trg': ['David', 'Gallo', ':', 'This', 'is', 'Bill', 'Lange', '.', 'I', "'m", 'Dave', 'Gallo', '.']}


In [0]:
# build the vocabulary
SRC.build_vocab(data_train, min_freq=2)
TRG.build_vocab(data_train, min_freq=2)

print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (de) vocabulary: 59346
Unique tokens in target (en) vocabulary: 36619


In [0]:
# BATCH_SIZE = 8

# train_iterator, valid_iterator = BucketIterator.splits(
#     (data_train, data_dev), 
#     batch_size = BATCH_SIZE, 
#     device = device)

In [0]:
def conv_to_words(sent, voc):
  sent_conv = []
  for idx in sent:
    sent_conv.append(voc.vocab.itos[idx])
  return sent_conv

def conv_to_idx(sent, voc):
  sent_conv = []
  for word in sent:
    sent_conv.append(voc.vocab.stoi[word])
  return sent_conv

PAD_IDX = 1

In [0]:
# # some outputs
# for i, batch in enumerate(train_iterator):
#   # print(i)
#   print(batch.src[:, 0])
#   print(conv_to_words(batch.src[:, 0], SRC))
#   print('SRC length: ', len(batch.src[:, 0]))
#   print(batch.trg[:, 0])
#   print(conv_to_words(batch.trg[:, 0], TRG))
#   print('TRG length: ', len(batch.trg[:, 0]))
#   break

In [0]:
# # more sentence printing
# new_s1 = ['<bos>', 'hi', 'my', 'name', 'is', 'david', '.', '<eos>', '<pad>']
# new_s2 = ['<bos>', 'how', 'are', 'you', 'doing', ',', 'David', '<unk>', '?', '<eos>', '<pad>', '<pad>', 'sadgahdj']
# print(conv_to_idx(new_s1, TRG))
# print(conv_to_idx(new_s2, TRG))

In [0]:
# build model
class EncDec(nn.Module):
  def __init__(self, encoder, decoder, s_embed, t_embed, gen):
    super(EncDec, self).__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.s_embed = s_embed
    self.t_embed = t_embed
    self.gen = gen
    
  def forward(self, src, trg=None, src_mask=None, trg_mask=None):
    return self.decode(self.encode(src, src_mask), src_mask, trg, trg_mask)
  
  def encode(self, src, src_mask):
    return self.encoder(self.s_embed(src), src_mask)
  
  def decode(self, memory, src_mask, trg, trg_mask):
    return self.decoder(self.t_embed(trg), memory, src_mask, trg_mask)
  
class Generator(nn.Module):
  # linear and softmax generation
  def __init__(self, d_model, vocab):
    super(Generator, self).__init__()
    self.proj = nn.Linear(d_model, vocab)
    
  def forward(self, x):
    return F.log_softmax(self.proj(x), dim=-1)
  
def clones(module, N):
  return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

class Encoder(nn.Module):
  # transformer encoder layer
  # pass input through each layer and apply layer norm to the final output
  def __init__(self, layer, N):
    super(Encoder, self).__init__()
    self.layers = clones(layer, N)
    self.norm = LayerNorm(layer.size)
    
  def forward(self, x, mask):
    # print('enc x: ', x.size())
    # print('enc mask: ', mask.size())
    for layer in self.layers:
      x = layer(x, mask)
    return self.norm(x)
  
class LayerNorm(nn.Module):
  # applying layer normalization to the input
  def __init__(self, features, eps=1e-6):
    super(LayerNorm, self).__init__()
    self.a_2 = nn.Parameter(torch.ones(features))
    self.b_2 = nn.Parameter(torch.zeros(features))
    self.eps = eps
    
  def forward(self, x):
    mean = x.mean(-1, keepdim=True)
    std = x.std(-1, keepdim=True)
    return self.a_2 * (x - mean) / (std + self.eps) + self.b_2
  
class SubLayerConnect(nn.Module):
  def __init__(self, size, dropout):
    super(SubLayerConnect, self).__init__()
    self.norm = LayerNorm(size)
    self.dropout = nn.Dropout(dropout)
    
  def forward(self, x, sublayer):
    # print('sublayer x: ', x.size())
    # temp1 = self.norm(x)
    # print('temp1: ', temp1.size())
    # temp2 = sublayer(temp1)
    # print('temp2: ', temp2.size())
    # temp3 = self.dropout(temp2)
    # print('x: ', x.size())
    # print('temp3: ', temp3.size())
    # temp4 = x + temp3
    
    return x + self.dropout(sublayer(self.norm(x)))
  
class EncoderLayer(nn.Module):
  # self-attention layer followed by a feed-forward layer
  def __init__(self, size, self_attn, feedfwd, dropout):
    super(EncoderLayer, self).__init__()
    self.self_attn = self_attn
    self.feedfwd = feedfwd
    self.sublayer = clones(SubLayerConnect(size, dropout), 2)
    self.size = size
    
  def forward(self, x, mask):
    # print('enc layer x: ', x.size())
    # print('enc layer mask: ', mask.size())
    # self_attn thrice as per the paper
    x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
    # now apply feed forward layer
    return self.sublayer[1](x, self.feedfwd)
  
class Decoder(nn.Module):
  def __init__(self, layer, N):
    super(Decoder, self).__init__()
    self.layers = clones(layer, N)
    self.norm = LayerNorm(layer.size)
    
  def forward(self, x, memory, src_mask, trg_mask):
    # print('dec x: ', x.size())
    # print('dec memory: ', memory.size())
    # print('dec src_mask: ', src_mask.size())
    # print('dec trg_mask: ', trg_mask.size())
    for layer in self.layers:
      x = layer(x, memory, src_mask, trg_mask)
    return self.norm(x)
  
class DecoderLayer(nn.Module):
  # two multi-head attention layers (self attention and source attention) followed by a feed forward layer
  def __init__(self, size, self_attn, src_attn, feedfwd, dropout):
    super(DecoderLayer, self).__init__()
    self.size = size
    self.self_attn = self_attn
    self.src_attn = src_attn
    self.feedfwd = feedfwd
    self.sublayer = clones(SubLayerConnect(size, dropout), 3)
    
  def forward(self, x, memory, src_mask, trg_mask):
    # print('dec layer x: ', x.size())
    # print('dec layer memory: ', memory.size())
    # print('dec layer src_mask: ', src_mask.size())
    # print('dec layer trg_mask: ', trg_mask.size())
    m = memory
    # self-attention layer
    # print('here 1')
    x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, trg_mask))
    # source-attention layer
    # print('here 2')
    # print('x: ', x.size())
    # print('m: ', m.size())
    # print('src_mask: ', src_mask.size())
    x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
    # feedforward layer
    # print('here 3')
    return self.sublayer[2](x, self.feedfwd)
  
def subsequent_mask(size):
  # to prevent looking at future positions
  attn_shape = (1, size, size)
  sub_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
  # print('sub_mask: ', sub_mask.size())
  return torch.from_numpy(sub_mask) == 0

def attention(query, key, value, mask=None, dropout=None):
  # scaled dot product attention
  d_k = query.size(-1)
  # print('query size: ', query.size())
  # print('key size: ', key.size())
  scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
  # print('scores size: ', scores.size())
  # print('mask size: ', mask.size())
  if mask is not None:
    # mask = mask.transpose(2, 3)
    scores = scores.masked_fill(mask == 0, -1e9)
  p_attn = F.softmax(scores, dim=-1)
  if dropout is not None:
    p_attn = dropout(p_attn)
  # print('value size: ', value.size())
  # temp4 = torch.matmul(p_attn, value)
  # print('temp4: ', temp4.size())
  # print('p_attn: ', p_attn.size())
  return torch.matmul(p_attn, value), p_attn

class MultiHeadAttn(nn.Module):
  def __init__(self, h, d_model, dropout=0.1):
    super(MultiHeadAttn, self).__init__()
    assert d_model % h == 0
    self.d_k = d_model // h
    self.h = h
    self.linears = clones(nn.Linear(d_model, d_model), 4)
    self.attn = None
    self.dropout = nn.Dropout(p=dropout)
    
  def forward(self, query, key, value, mask=None):
    if mask is not None:
      mask = mask.unsqueeze(1)
      
    nbatches = query.size(0)
    
    # linear projections
    query, key, value = [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
                          for l, x in zip(self.linears, (query, key, value))]
    
    # apply attention on all projected vectors
    x, self.attn = attention(query, key, value, mask=mask, dropout=self.dropout)
    
    # concatenate
    x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)
    
    return self.linears[-1](x)

class PosFeedFwd(nn.Module):
  def __init__(self, d_model, d_ff, dropout=0.1):
    super(PosFeedFwd, self).__init__()
    self.w_1 = nn.Linear(d_model, d_ff)
    self.w_2 = nn.Linear(d_ff, d_model)
    self.dropout = nn.Dropout(dropout)
    
  def forward(self, x):
    return self.w_2(self.dropout(F.relu(self.w_1(x))))
  
class Embeddings(nn.Module):
  def __init__(self, d_model, vocab):
    super(Embeddings, self).__init__()
    self.lut = nn.Embedding(vocab, d_model)
    self.d_model = d_model
    
  def forward(self, x):
    return self.lut(x) * math.sqrt(self.d_model)
  
class PosEncoding(nn.Module):
  def __init__(self, d_model, dropout, max_len=1000):
    super(PosEncoding, self).__init__()
    self.dropout = nn.Dropout(p=dropout)
    # compute the encodings in log-space
    pe = torch.zeros(max_len, d_model)
    position = torch.arange(0, max_len).unsqueeze(1).float()
    div_term = torch.exp(torch.arange(0.0, d_model, 2) * -(math.log(10000.0) / d_model)).float()
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    pe = pe.unsqueeze(0)
    self.register_buffer('pe', pe)
    
  def forward(self, x):
    x = x + Variable(self.pe[:, :x.size(1)], requires_grad=False)
    return self.dropout(x)
  
def make_model(src_vocab, trg_vocab, N=3, d_model=256, d_ff=512, h=2, dropout=0.1):
  c = copy.deepcopy
  attn = MultiHeadAttn(h, d_model)
  ff = PosFeedFwd(d_model, d_ff, dropout)
  position = PosEncoding(d_model, dropout)
  model = EncDec(
            Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
            Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
            nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
            nn.Sequential(Embeddings(d_model, trg_vocab), c(position)),
            Generator(d_model, trg_vocab))
  
  for p in model.parameters():
    if p.dim() > 1:
      nn.init.xavier_uniform_(p)
      
  return model
    

In [0]:
class Batch:
  def __init__(self, src, trg=None, pad=1):
    self.src = src
    self.src_mask = (src != pad).unsqueeze(-2)
    if trg is not None:
      self.trg = trg[:, :-1]
      self.trg_y = trg[:, 1:]
      self.trg_mask = self.make_std_mask(self.trg, pad)
      self.ntokens = (self.trg_y != pad).data.sum()
      
  @staticmethod 
  def make_std_mask(trg, pad):
    trg_mask = (trg != pad).unsqueeze(-2)
    # trg_len = trg.shape[1]
    # trg_sub_mask = torch.triu(torch.ones((trg_len, trg_len), dtype=torch.uint8)).cuda()
    # trg_mask = trg_mask & trg_sub_mask
    trg_mask = trg_mask & Variable(subsequent_mask(trg.size(1)).type_as(trg_mask.data)).cuda()
    # tgt_mask = tgt_mask[:, :, 0]
    return trg_mask
      
def run_epoch(data_iter, model, loss_compute):
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    for i, batch in enumerate(data_iter):
        out = model.forward(batch.src, batch.trg, 
                            batch.src_mask, batch.trg_mask)
        loss = loss_compute(out, batch.trg_y, batch.ntokens)
        total_loss += loss
        total_tokens += batch.ntokens
        tokens += batch.ntokens
        if i % 50 == 1:
            elapsed = time.time() - start
            print("Epoch Step: %d Loss: %f Tokens per Sec: %f" %
                    (i, loss / batch.ntokens, tokens / elapsed))
            start = time.time()
            tokens = 0
    return total_loss / total_tokens
  
global max_src_in_batch, max_tgt_in_batch
def batch_size_fn(new, count, sofar):
    global max_src_in_batch, max_tgt_in_batch
    if count == 1:
        max_src_in_batch = 0
        max_tgt_in_batch = 0
    max_src_in_batch = max(max_src_in_batch,  len(new.src))
    max_tgt_in_batch = max(max_tgt_in_batch,  len(new.trg) + 2)
    src_elements = count * max_src_in_batch
    tgt_elements = count * max_tgt_in_batch
    return max(src_elements, tgt_elements)

In [0]:
# optimizer
class NoamOpt:
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))
        
def get_std_opt(model):
    return NoamOpt(model.src_embed[0].d_model, 2, 4000,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

In [0]:
class LabelSmoothing(nn.Module):
    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(size_average=False)
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None
        
    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, Variable(true_dist, requires_grad=False))
      
class SimpleLossCompute:
    "A simple loss compute and train function."
    def __init__(self, generator, criterion, opt=None):
        self.generator = generator
        self.criterion = criterion
        self.opt = opt
        
    def __call__(self, x, y, norm):
        x = self.generator(x)
        loss = self.criterion(x.contiguous().view(-1, x.size(-1)), 
                              y.contiguous().view(-1)) / norm
        loss.backward()
        if self.opt is not None:
            self.opt.step()
            self.opt.optimizer.zero_grad()
        return loss.data[0] * norm

In [0]:
# # for our overfit model 
# model = make_model(len(SRC.vocab), len(TRG.vocab))
# model.cuda()
# def count_parameters(model):
#   return sum(p.numel() for p in model.parameters() if p.requires_grad)

# print(f'The model has {count_parameters(model):,} trainable parameters')

# BATCH_SIZE = 4
# train_iterator, valid_iterator = BucketIterator.splits(
#     (data_train, data_dev), 
#     batch_size = BATCH_SIZE, 
#     device = device)

# for i, batch in enumerate(train_iterator):
#   # print(batch.src)
#   # print(batch.trg)
# #   print(i)
#   if i == 1:
#     overfit_src = batch.src.transpose(1, 0)
#     overfit_trg = batch.trg.transpose(1, 0)
#     break

# # overfit_src = overfit_src[:50, :]
# # overfit_trg = overfit_trg[:55, :]
# print(overfit_src.size())
# print(overfit_trg.size())
 

In [0]:
# # data for overfitting
# of = Batch(overfit_src, overfit_trg, pad=1)

# print(of.src.size())
# # print(of.src[:, 0])
# print(of.src_mask.size())
# # print(of.src_mask[:, 0])
# print(of.trg.size())
# # print(of.trg[:, 0])
# print(of.trg_mask.size())
# # print(of.trg_mask[:, 0])

In [0]:
# # overfit on the training data
# optimizer = NoamOpt(256, 1, 800, torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))
# # criterion = nn.CrossEntropyLoss(ignore_index=1)
# criterion = LabelSmoothing(size=len(TRG.vocab), padding_idx=1, smoothing=0.0)
# def o_train(model, optimizer, criterion, clip, batch, batch_size):
#   model.train()
#   total_loss = 0
#   total_iters = int(batch.src.size(0) / batch_size)
#   #print(total_iters)
#   sz = batch_size
#   for i in range(total_iters):
#     tb_src = batch.src[sz-batch_size:sz, :]
#     tb_src_mask = batch.src_mask[sz-batch_size:sz, :, :]
#     tb_trg = batch.trg[sz-batch_size:sz, :]
#     ground_trg = tb_trg
#     # shift the targets by one
#     tb_trg_extra = tb_trg[:, -1].unsqueeze(1)
#     tb_trg = tb_trg[:, :-1]
#     tb_trg = torch.cat([tb_trg_extra, tb_trg], dim=1)
#     tb_trg_mask = batch.trg_mask[sz-batch_size:sz, :, :]
#     sz = sz + batch_size
#     # print(tb_trg[0])
#     # print(tb_trg_mask[0])
#     # break
#     # print(sz)
#     # print(tb_src_mask[0])
#     # print(tb_src.size(), tb_trg.size(), tb_src_mask.size(), tb_trg_mask.size())
#     #tb_trg1 = torch.Tensor(234 * torch.ones((4, 84))).long().cuda()
#     optimizer.optimizer.zero_grad()
#     # out = model.forward(tb_src, tb_trg, tb_src_mask, tb_trg_mask)
#     out = model.encode(tb_src, tb_src_mask)
#     out1 = model.decode(out, tb_src_mask, tb_trg, tb_trg_mask)
#     prob = model.gen(out1)
# #     print('prob: ', prob.size())
#     if i == 1:
#       _, tmp_pred = torch.max(prob, dim=2)
#       print('pred: ', conv_to_words(tmp_pred[0], TRG))
#       print('ground: ', conv_to_words(ground_trg[0], TRG))
#     #print(prob.size(), tb_trg.size())
# #     break
#     prob = prob.contiguous().view(-1, prob.shape[-1])
#     ground_trg = ground_trg.contiguous().view(-1)
#     # print(prob.size(), ground_trg.size())
#     loss = criterion(prob, ground_trg)
#     loss.backward()
#     torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
#     optimizer.step()
#     total_loss += loss.item()
    
#   return total_loss / total_iters

# num_epochs = 50
# clip = 1
# for epoch in range(num_epochs):
#   start_time = time.time()
#   train_loss = o_train(model, optimizer, criterion, clip, of, 4)
#   end_time = time.time()
#   elapsed_time = int(end_time - start_time) / 60
#   print('Epoch: ', epoch, ' Time: ', elapsed_time, ' Loss: ', train_loss)
#   # break
  
  

In [0]:
# # greedy decoding 
# model.eval()
# t_src = of.src[1:3, :]
# t_src_mask = of.src_mask[1:3, :]
# t_trg = of.trg[1:3, :]
# t_trg_mask = of.trg_mask[1:3, :]
# start_symbol = 2
# # print(t_src.size())
# # print(t_src_mask.size())
# # print(t_trg.size())
# # print(t_trg_mask.size())
# # print(conv_to_words(t_src[0, :], SRC))
# # print(conv_to_words(t_trg[0, :], TRG))
# # print(conv_to_words(t_src[1, :], SRC))
# # print(conv_to_words(t_trg[1, :], TRG))
                 
# memory = model.encode(t_src, t_src_mask)
# # print('memory: ', memory.size())
# # out1 = model.decode(memory, t_src_mask, tmp_trg, tmp_trg_mask)
# # out2 = model.gen(out1[:, -1])
# # print('out2: ', out2.size())
# # print('1: ', torch.argmax(out2, dim=1))
# # print(out2[0])
# # _, next_word = torch.max(out2, dim=1)
# # next_word = next_word.data
# # print(next_word[0])
# # print(memory)
# tmp_trg = Variable(torch.zeros(2, 30).type_as(t_src.data)).cuda()
# next_word = start_symbol
# # print(tmp_trg.size())
# for i in range(0, 30):
# #   if i != 0:
# #     next_word = prob.data[i]
# #     next_word = next_word.item()
#   tmp_trg[:, i] = next_word
#   tmp_trg_mask = Variable(subsequent_mask(tmp_trg.size(1)).unsqueeze(0).type_as(t_src.data)).cuda()
#   out = model.decode(memory, t_src_mask, tmp_trg, tmp_trg_mask)
#   out2 = model.gen(out)
#   # print(out2.size())
#   prob = out2.max(dim=-1, keepdim=False)[1]
#   # print(prob)
#   # break
#   # print(out2.size())
#   # _, next_word = torch.max(out2, dim=2)
#   # print(next_word[0][i+1])
#   # tmp_max = torch.argmax(out2, dim=1)
#   # print(tmp_max)
#   # print(out2.size())
#   _, tmp_topk = torch.topk(out2, 3, dim=-1)
#   # print(tmp_topk[:, i])
# #   print(t_trg[0][i].unsqueeze(0))
#   next_word = prob.data[:, i].unsqueeze(0)
#   # print(next_word)
#   # next_word = next_word.item()
  
  
#   # break
#   # next_word = Variable(next_word[0][i+1].unsqueeze(0).type_as(t_src.data)).cuda()
#   # print(next_word, tmp_trg)
#   # break
#   # tmp_trg = torch.cat([tmp_trg, t_trg[0][i].unsqueeze(0).unsqueeze(1)], dim=1)
  
  
# print(tmp_trg)
# print(t_trg)
# print('pred: ', conv_to_words(tmp_trg[0], TRG))
# print('ground: ', conv_to_words(t_trg[0], TRG))
# print('pred: ', conv_to_words(tmp_trg[1], TRG))
# print('ground: ', conv_to_words(t_trg[1], TRG))

In [0]:
# # decoding using ground truths
# model.eval()
# t_src = of.src[1:2, :]
# t_src_mask = of.src_mask[1:2, :]
# t_trg = of.trg[1:2, :]
# t_trg_mask = of.trg_mask[1:2, :]
# start_symbol = 2
# # print(t_src.size())
# # print(t_src_mask.size())
# # print(t_trg.size())
# # print(t_trg_mask.size())
# # print(conv_to_words(t_src[0, :], SRC))
# # print(conv_to_words(t_trg[0, :], TRG))
# # print(conv_to_words(t_src[1, :], SRC))
# # print(conv_to_words(t_trg[1, :], TRG))
                 
# memory = model.encode(t_src, t_src_mask)
# # print('memory: ', memory.size())
# # out1 = model.decode(memory, t_src_mask, tmp_trg, tmp_trg_mask)
# # out2 = model.gen(out1[:, -1])
# # print('out2: ', out2.size())
# # print('1: ', torch.argmax(out2, dim=1))
# # print(out2[0])
# # _, next_word = torch.max(out2, dim=1)
# # next_word = next_word.data
# # print(next_word[0])
# # print(memory)
# tmp_trg = Variable(torch.ones(1, 1).fill_(start_symbol).type_as(t_src.data)).cuda()
# # print(tmp_trg.size())
# for i in range(20):
#   tmp_trg_mask = Variable(subsequent_mask(tmp_trg.size(1)).unsqueeze(0).type_as(t_src.data)).cuda()
#   out = model.decode(memory, t_src_mask, tmp_trg, tmp_trg_mask)
#   out2 = model.gen(out[:, -1])
#   _, next_word = torch.max(out2, dim=1)
#   # print(next_word)
#   # tmp_max = torch.argmax(out2, dim=1)
#   # print(tmp_max)
#   _, tmp_topk = torch.topk(out2, 7, dim=1)
#   print(tmp_topk)
# #   print(t_trg[0][i].unsqueeze(0))
#   next_word = Variable(next_word.unsqueeze(0).type_as(t_src.data)).cuda()
#   tmp_trg = torch.cat([tmp_trg, t_trg[0][i].unsqueeze(0).unsqueeze(1)], dim=1)
  
  
# print(tmp_trg)
# print(t_trg)
# print(conv_to_words(tmp_trg[0], TRG))
# # print(conv_to_words(tmp_trg[1], TRG))
# print(conv_to_words(t_trg[0], TRG))
# # print(conv_to_words(t_trg[1], TRG))

In [0]:
# train the model
# model = make_model(len(SRC.vocab), len(TRG.vocab))
# load the temp model
temp_path = 'tf1_temp10.pth'
temp_id = '17290deLOCbGqAsYQob46R0up1Gt_2abp'
to_load = drive.CreateFile({"id": temp_id}) # point to the correct file using its id
to_load.GetContentFile(temp_path) # copies the model from drive to colab machine
model = torch.load(temp_path)
model.cuda()
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

BATCH_SIZE = 8
train_iterator, valid_iterator = BucketIterator.splits(
    (data_train, data_dev), 
    batch_size = BATCH_SIZE, 
    device = device)

optimizer = NoamOpt(256, 1, 4000, torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))
criterion = LabelSmoothing(size=len(TRG.vocab), padding_idx=1, smoothing=0.0)

def train(model, optimizer, criterion, clip, batch, i):
  model.train()
  total_loss = 0
  tb_src = batch.src
  tb_src_mask = batch.src_mask
  tb_trg = batch.trg
  ground_trg = tb_trg
  # shift the targets by one
  tb_trg_extra = tb_trg[:, -1].unsqueeze(1)
  tb_trg = tb_trg[:, :-1]
  tb_trg = torch.cat([tb_trg_extra, tb_trg], dim=1)
  tb_trg_mask = batch.trg_mask
  optimizer.optimizer.zero_grad()
  out = model.encode(tb_src, tb_src_mask)
  out1 = model.decode(out, tb_src_mask, tb_trg, tb_trg_mask)
  prob = model.gen(out1)
  if i == 24000:
    _, tmp_pred = torch.max(prob, dim=2)
    print('\n Training:')
    print('pred: ', conv_to_words(tmp_pred[0], TRG), 'len: ', len(tmp_pred[0]))
    print('ground: ', conv_to_words(ground_trg[0], TRG), 'len: ', len(ground_trg[0]))
  prob = prob.contiguous().view(-1, prob.shape[-1])
  ground_trg = ground_trg.contiguous().view(-1)

  loss = criterion(prob, ground_trg)
  loss.backward()
  torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
  optimizer.step()
  total_loss += loss.item()
    
  return total_loss 

def eval(model, criterion, batch, i):
  model.eval()
  total_loss = 0
  vb_src = batch.src
  vb_src_mask = batch.src_mask
  vb_trg = batch.trg
  ground_trg = vb_trg
  # shift the targets by one
  vb_trg_extra = vb_trg[:, -1].unsqueeze(1)
  vb_trg = vb_trg[:, :-1]
  vb_trg = torch.cat([vb_trg_extra, vb_trg], dim=1)
  vb_trg_mask = batch.trg_mask

  out = model.encode(vb_src, vb_src_mask)
  out1 = model.decode(out, vb_src_mask, vb_trg, vb_trg_mask)
  prob = model.gen(out1)
  if i == 4000:
    print('\n Eval:')
    _, tmp_pred = torch.max(prob, dim=2)
    print('pred: ', conv_to_words(tmp_pred[0], TRG), 'len: ', len(tmp_pred[0]))
    print('ground: ', conv_to_words(ground_trg[0], TRG), 'len: ', len(ground_trg[0]))
  prob = prob.contiguous().view(-1, prob.shape[-1])
  ground_trg = ground_trg.contiguous().view(-1)

  loss = criterion(prob, ground_trg)
  total_loss += loss.item()
    
  return total_loss 

num_epochs = 30
clip = 1
print(len(train_iterator))
for epoch in range(num_epochs):
  start_time = time.time()
  train_loss = 0
  val_loss = 0
  for i, batch in enumerate(train_iterator):
    if i % 8000 == 0:
      int_time = time.time()
      el_time = int(int_time - start_time) / 60
      print(i, 'Time: ', el_time)
    t_src = batch.src.transpose(1, 0)
    t_trg = batch.trg.transpose(1, 0)
    t_batch = Batch(t_src, t_trg, pad=1)
    train_loss += train(model, optimizer, criterion, clip, t_batch, i)
    
  train_loss = train_loss / len(train_iterator)
  
  for i, batch in enumerate(valid_iterator):
    v_src = batch.src.transpose(1, 0)
    v_trg = batch.trg.transpose(1, 0)
    v_batch = Batch(v_src, v_trg, pad=1)
    val_loss += eval(model, criterion, v_batch, i)
    
  val_loss = val_loss / len(valid_iterator)
    
  end_time = time.time()
  elapsed_time = int(end_time - start_time) / 60
  print('Epoch: ', epoch, ' Time: ', elapsed_time, ' Train Loss: ', train_loss, 'Val Loss: ', val_loss)
  # save the partial model
  if epoch % 2 == 0:
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    print('success!')
    file_name = 'tf2_temp' + str(epoch) + '.pth'
    torch.save(model, file_name)
    model_file = drive.CreateFile()
    model_file.SetContentFile(file_name) # will copy the file from colab machine to google drive
    model_file.Upload()
    print(model_file['id'])


The model has 37,932,811 trainable parameters
24611


/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


0 Time:  0.0
8000 Time:  8.35
16000 Time:  16.7
24000 Time:  25.05

 Training:
pred:  ['<bos>', 'But', 'you', 'at', 'the', 'number', 'of', 'black', 'lines', 'in', 'this', 'results', '.', '<eos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>'] len:  40
ground:  ['<bos>', 'But', 'look', 'at', 'the', 'number', 'of', 'black', 'lines', 'in', 'that', 'spectrum', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>'] len:  40
Epoch:  0  Time:  25.916666666666668  Train Loss:  647.1756068207936 Val Loss:  556.5000221488684
success!


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type EncDec. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Encoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type EncoderLayer. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type MultiHeadAttn. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/pyt

1NPRuHURYgMS1WmCiUPVl0AW9bLBt-MZC
0 Time:  0.0
8000 Time:  8.383333333333333
16000 Time:  16.766666666666666
24000 Time:  25.083333333333332

 Training:
pred:  ['<bos>', 'That', 'that', 'means', 'that', '<unk>', ',', 'that', 'have', 'to', 'be', 'interesting', 'to', 'interesting', '.', 'interesting', '.', '<eos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>', '<bos>'] len:  32
ground:  ['<bos>', 'And', 'that', 'means', 'visual', 'intelligence', 'software', 'will', 'need', 'to', 'scan', 'it', 'for', 'items', 'of', 'interest', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>'] len:  32
Epoch:  1  Time:  25.95  Train Loss:  609.1946354440128 Val Loss:  546.2040060122647
0 Time:  0.0
8000 Time:  8.366666666666667
16000 Time:  16.75
24000 Time:  25.1

 Training:
pred:  ['<bos>', 'We', 'long', 'as', ',', 'we', 'can', 'see', 'back', 'we

KeyboardInterrupt: ignored

In [0]:
# save the final model
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
print('success!')
torch.save(model, 'tf3_final.pth')
model_file = drive.CreateFile()
model_file.SetContentFile('tf3_final.pth') # will copy the file from colab machine to google drive
model_file.Upload()
print(model_file['id'])

success!


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type EncDec. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Encoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type EncoderLayer. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type MultiHeadAttn. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/pyt

1xMxiYu-Xi8z-SJSNmKuNLrADsIawc2em


In [0]:
!nvidia-smi

In [0]:
# compute the BLEU for the val set without using iterators
temp_path = 'tf3_final.pth'
temp_id = '1xMxiYu-Xi8z-SJSNmKuNLrADsIawc2em'
to_load = drive.CreateFile({"id": temp_id}) # point to the correct file using its id
to_load.GetContentFile(temp_path) # copies the model from drive to colab machine
model = torch.load(temp_path)
model.cuda()
model.eval()
file1 = open('val_pred_t.en', 'w')
# print(data_dev.src)
# for i, batch in enumerate(data_dev):
  
#   print(batch.src)
#   print(batch.trg)

for i, batch in enumerate(data_dev):
  v_src = batch.src
  v_trg = batch.trg
  v_src = conv_to_idx(v_src, SRC)
  v_trg = conv_to_idx(v_trg, TRG)
  
  v_src = torch.LongTensor(v_src).unsqueeze(1).cuda()
  v_trg = torch.LongTensor(v_trg).unsqueeze(1).cuda()
  
  
  # print(v_src, v_trg)
  v_src = v_src.transpose(1, 0)
  v_trg = v_trg.transpose(1, 0).squeeze(0)
  start_t = torch.LongTensor(conv_to_idx(['<bos>'], TRG)).cuda()
  # print(v_trg.size(), start_t.size())
  v_trg = torch.cat([start_t, v_trg], dim=0).unsqueeze(0)
  v_batch = Batch(v_src, v_trg, pad=1)
  vb_src = v_batch.src
  vb_src_mask = v_batch.src_mask
  vb_trg = v_batch.trg
  
  # print(start_t)
  # print(vb_trg.size(), start_t.size())
  
  ground_trg = vb_trg
  # shift the targets by one
  vb_trg_extra = vb_trg[:, -1].unsqueeze(1)
  vb_trg = vb_trg[:, :-1]
  vb_trg = torch.cat([vb_trg_extra, vb_trg], dim=1)
  vb_trg_mask = v_batch.trg_mask

  out = model.encode(vb_src, vb_src_mask)
  out1 = model.decode(out, vb_src_mask, vb_trg, vb_trg_mask)
  prob = model.gen(out1)
  # print(prob.size())
  _, tmp_pred = torch.max(prob, dim=2)
  # print(tmp_pred)
  
  for j in range(len(tmp_pred)):
    sent1 = ''
    # print('pred: ', conv_to_words(tmp_pred[j], TRG))
    # print('ground: ', conv_to_words(ground_trg[j], TRG))
    # print('\n')
    # remove bos, append until eos and save the generated text
    word_pred = conv_to_words(tmp_pred[j], TRG)
    word_ground = conv_to_words(ground_trg[j], TRG)
    for k in range(len(word_pred)):
      if word_pred[k] == '<eos>':
        break
      elif word_pred[k] == '<bos>':
        continue
      sent1 = sent1 + str(word_pred[k]) + ' '
    
    sent1 = sent1 + '\n'
    print(sent1)
    # print(1)
    
    file1.write(sent1)
  
file1.close()


When I was in my ' , I had my first first <unk> . 

I was a student student and college psychology at No 

She was a <unk> - old woman named <unk> 

And , first in the first session , the , a top <unk> , , and she fell on the house in my office , her her her her of told me she was there to talk about men 's 

And , I heard that , I was so going 

Because first got a <unk> a the first patient 

And I got a woman woman wanted to talk about boys 

, I is thought I 'd . 

But I did n't get it 

And the fun stories that the came bring in the , it was easy to me to to talk to head down we 're the problems of the problems 

now " 30 's the new 20 , " I said say , and as far as I know , you she was right 

, <unk> came , , married came , , kids came later , even the came later 

like People like the and I did nothing to my 

soon but soon I , my the , me to ask my and a question question 

. I thought on 

I said , " Yes , she 's going to to with 's going with men <unk> , but she 's not going 

In [0]:
!ls
file1 = open('val_pred_t.en', 'r')
file2 = open('dev.en', 'r')
for i in range(500):
  sent1 = file1.readline()
  sent2 = file2.readline()
  print('pred: ', sent1)
  print('gold: ', sent2)
  print('\n')

adc.json	 sample_data	 tf2_temp16.pth  tf3_final.pth
dev1.en		 tf1_final1.pth  tf2_temp18.pth  train.de
dev.de		 tf1_temp10.pth  tf2_temp20.pth  train.en
dev.en		 tf2_temp0.pth	 tf2_temp2.pth	 val_pred.en
dev_en1		 tf2_temp10.pth  tf2_temp4.pth	 val_pred.en.detok
iwslt_en_de.zip  tf2_temp12.pth  tf2_temp6.pth	 val_pred_t.en
mosesdecoder	 tf2_temp14.pth  tf2_temp8.pth
pred:  When I was in my ' , I had my first first <unk> . 

gold:  When I was in my 20s, I saw my very first psychotherapy client.



pred:  I was a student student and college psychology at No 

gold:  I was a Ph.D. student in clinical psychology at Berkeley.



pred:  She was a <unk> - old woman named <unk> 

gold:  She was a 26-year-old woman named Alex.



pred:  And , first in the first session , the , a top <unk> , , and she fell on the house in my office , her her her her of told me she was there to talk about men 's 

gold:  Now Alex walked into her first session wearing jeans and a big slouchy top, and she dropped 

In [0]:
!ls
!git clone https://github.com/moses-smt/mosesdecoder.git
!pip install sacrebleu
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
print('Authentication Successful!')
iwslt_data = drive.CreateFile({'id': '1Swpb8yG3atDzxiOR0dQuFE-3KjgZCuWE'})
iwslt_data.GetContentFile('iwslt_en_de.zip')

!unzip iwslt_en_de.zip dev.en

adc.json  dev.de  dev.en  sample_data  train.de  train.en
Cloning into 'mosesdecoder'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 147302 (delta 0), reused 0 (delta 0), pack-reused 147295
Receiving objects: 100% (147302/147302), 129.69 MiB | 24.25 MiB/s, done.
Resolving deltas: 100% (113834/113834), done.
  Stored in directory: /root/.cache/pip/wheels/40/f1/a9/4cce0ec602e8d195da27bb9b8f6708ec778fbafdbabb097fde
Successfully built sacrebleu
Authentication Successful!
Archive:  iwslt_en_de.zip
replace dev.en? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dev.en                  


In [0]:
%%shell
#!/bin/bash

# This is a reference to the gold translations from the dev set
REFERENCE_FILE="dev.en"

# XXX: Change the following line to point to your model's output!
TRANSLATED_FILE="val_pred_t.en"

# The model output is expected to be in a tokenized form. Note, that if you
# tokenized your inputs to the model, then simply joined each output token with
# whitespace you should get tokenized outputs from your model.
# i.e. each output token is separate by whitespace
# e.g. "My model 's output is interesting ."
perl "mosesdecoder/scripts/tokenizer/detokenizer.perl" -l en < "$TRANSLATED_FILE" > "$TRANSLATED_FILE.detok"

PARAMS=("-tok" "intl" "-l" "de-en" "$REFERENCE_FILE")
sacrebleu "${PARAMS[@]}" < "$TRANSLATED_FILE.detok"

Detokenizer Version $Revision: 4134 $
Language: en
BLEU+case.mixed+lang.de-en+numrefs.1+smooth.exp+tok.intl+version.1.3.2 = 22.3 53.2/28.2/16.5/10.0 (BP = 1.000 ratio = 1.034 hyp_len = 166392 ref_len = 160956)


In [0]:
# upload translations to drive
tr_file = drive.CreateFile()
tr_file.SetContentFile('val_pred_t.en') # will copy the file from colab machine to google drive
tr_file.Upload()
print(tr_file['id'])

FileNotFoundError: ignored

In [0]:
!ls
!cp test.de test.en

adc.json  dev.en	   mosesdecoder  test.de       train.de
dev.de	  iwslt_en_de.zip  sample_data	 test_pred.en  train.en


In [35]:
temp_path = 'tf3_final.pth'
temp_id = '1xMxiYu-Xi8z-SJSNmKuNLrADsIawc2em'
to_load = drive.CreateFile({"id": temp_id}) # point to the correct file using its id
to_load.GetContentFile(temp_path) # copies the model from drive to colab machine
model = torch.load(temp_path)
model.cuda()
model.eval()
# generate data for test set
# get the test data
link = 'https://drive.google.com/open?id=1DKIgsNhxq_YbXIRRnZjaxF8CVuVAEyHC'
fluff, id = link.split('=')
print(id)
f_tr_en = drive.CreateFile({'id': id})
f_tr_en.GetContentFile('./test.de')

file3 = open('test_pred.en', 'w')
data_test = datasets.TranslationDataset(
            path='./test', exts=('.en','.de'), fields=(TRG, SRC))


# run greedy decoding
for i, batch in enumerate(data_test):
  model.eval()
  
  start_symbol = 2
  v_src = batch.src
  v_src = conv_to_idx(v_src, SRC)
  
  v_src = torch.LongTensor(v_src).unsqueeze(1).cuda()
  
  
  # print(v_src, v_trg)
  v_src = v_src.transpose(1, 0)
  v_batch = Batch(v_src, pad=1)
  vb_src = v_batch.src
  vb_src_mask = v_batch.src_mask
  
  memory = model.encode(vb_src, vb_src_mask)
  tmp_trg = Variable(torch.zeros(1, 50).type_as(vb_src.data)).cuda()
  next_word = start_symbol
  for i in range(0, 50):
    tmp_trg[:, i] = next_word
    tmp_trg_mask = Variable(subsequent_mask(tmp_trg.size(1)).unsqueeze(0).type_as(vb_src.data)).cuda()
    out = model.decode(memory, vb_src_mask, tmp_trg, tmp_trg_mask)
    out2 = model.gen(out)
    prob = out2.max(dim=-1, keepdim=False)[1]
    _, tmp_topk = torch.topk(out2, 3, dim=-1)
    next_word = prob.data[:, i].unsqueeze(0)
  tmp_trg = tmp_trg.squeeze(0)
  # print(tmp_trg)
  tmp_trg = conv_to_words(tmp_trg, TRG)
  # print(tmp_trg)
  sent1 = ''
  for k in range(len(tmp_trg)):
    if tmp_trg[k] == '<eos>':
      break
    elif tmp_trg[k] == '<bos>':
      continue
    sent1 = sent1 + str(tmp_trg[k]) + ' '
    
  sent1 = sent1 + '\n'
  print(sent1)
  # print(1)
  file3.write(sent1) 
    
file3.close()              


1DKIgsNhxq_YbXIRRnZjaxF8CVuVAEyHC
My goal in this video , and the next to the next , it is , and the next , and the next is it in this video , it is it is it 's it 's , it , it , and the next one thing in this 

And one of the size of the Earth , " on Earth , " on Earth . 

And the solar system . And how we 're going to see , and how we 're going to see , and how we 're to give us will see , we 're going to see , and see , and the solar system , and the solar system 

If we do the galaxy and the universe , the universe , the universe , and the universe , and the universe , we make the galaxy and the universe , we make the universe , we do the universe , if we make it , if we make 

It 's almost impossible to imagine all this . 

But we 're going to least , we 're going to end , but we 're going to at least , but we 're going to least try to be at least we 're going to be trying to do it 's going to do it at least we 

Well , I think most of the video , which are looking at this video ,

In [36]:
# save the translations
tr_file = drive.CreateFile()
tr_file.SetContentFile('test_pred.en') # will copy the file from colab machine to google drive
tr_file.Upload()
print(tr_file['id'])

1uMN8hgiC7nR5yaJGT71ja5r0pw0JFGZI


In [0]:
# N_EPOCHS = 1
# CLIP = 1

# best_valid_loss = float('inf')

# for epoch in range(N_EPOCHS):
    
#     start_time = time.time()
    
#     train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
#     valid_loss = evaluate(model, valid_iterator, criterion)
    
#     end_time = time.time()
    
#     epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
#     if valid_loss < best_valid_loss:
#         best_valid_loss = valid_loss
#         file_name = 'temp-tf-model' + str(epoch) + '.pth'
#         torch.save(model, file_name)
#         temp_file = drive.CreateFile()
#         temp_file.SetContentFile(file_name) 
#         temp_file.Upload()
#         print('Epoch Num: ', epoch, ' File ID: ', temp_file['id'])
    
#     print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
#     print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
#     print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f} |  Val. BLEU: {math.exp(valid_bleu):7.3f}')

In [0]:
# # for resuming training of partially-trained models
# N_EPOCHS = 20
# CLIP = 1

# # loading the temp-trained file
# temp_path = ' '
# temp_id = ' '
# to_load = drive.CreateFile({"id": temp_id}) # point to the correct file using its id
# to_load.GetContentFile(temp_path) # copies the model from drive to colab machine
# model = torch.load(temp_path)

# for epoch in range(N_EPOCHS):
    
#     start_time = time.time()
    
#     train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
#     valid_loss = evaluate(model, valid_iterator, criterion)
    
#     end_time = time.time()
    
#     epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
#     if valid_loss < best_valid_loss:
#         best_valid_loss = valid_loss
#         file_name = 'temp-tf-model' + str(epoch + temp_state['epoch']) + '.pth'
#         torch.save(model, file_name)
#         temp_file = drive.CreateFile()
#         temp_file.SetContentFile(file_name) 
#         temp_file.Upload()
#         print('Epoch Num: ', epoch, ' File ID: ', temp_file['id'])
    
#     print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
#     print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
#     print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f} |  Val. BLEU: {math.exp(valid_bleu):7.3f}')

In [0]:
# # for saving the final model
# tf_name = 'tf_final.pth'
# torch.save(model, tf_name)
# model_file = drive.CreateFile()
# model_file.SetContentFile(tf_name)
# model_file.Upload()
# print(model_file['id'])

In [0]:
# # for loading the final model
# tf_id = ''
# to_load = drive.CreateFile({"id": tf_id}) # point to the correct file using its id
# to_load.GetContentFile('tf_final.pth') # copies the model from drive to colab machine
# model = torch.load('tf_final.pth')


In [0]:
!nvidia-smi